In [1]:
from pathlib import Path
import os
import time
os.chdir("..")

In [2]:
# !pip install numpy typer sympy 

## Compilation [REQUIRED]

In [3]:
# compile bin, it takes ~ 2 min

t1 = time.time()
bin_dir = Path("bin")
bin_dir.mkdir(exist_ok=True)
for mod_flag, suffix in [("", "2"), ("-DMOD3", "3")]:
    for src in ["search", "lift", "select", "select_modular"]:
        name = f"{src}{suffix}"
        exe = bin_dir / name
        !g++ {mod_flag} -Ofast -DNDEBUG -std=c++20 -march=native -Ithird_party -pthread src/{src}.cpp -o {exe}
        globals()[name] = exe
t2 = time.time()
print(f"Compiled in {t2-t1:.1f}s.")

Compiled in 124.8s.


In [4]:
# use -h to check parameters
!{search2} -h

Pool-based Tensor Flip Graph Search 


bin\search2 [OPTIONS] name


POSITIONALS:
  name TEXT REQUIRED          Tensor name (e.g., gt-333) 

OPTIONS:
  -h,     --help              Print this help message and exit 
          --id TEXT           Output identifier (affects scheme names only) 
  -f,     --path-limit INT [1000000]  
                              Path length limit 
  -s,     --pool-size INT [200]  
                              Pool size limit 
  -r,     --target-rank INT [0]  
                              Target rank 
  -p,     --plus-lim INT [50000]  
                              Flips before plus transition 
  -t,     --threads INT:POSITIVE [4]  
                              Number of worker threads 
  -m,     --max-attempts INT:POSITIVE [1000]  
                              Max attempts per rank level 
          --stop INT:POSITIVE [20000]  
                              Stop if nothing found after this many attempts 
          --plus              Enable plus transiti

## General matrix multiplication

### GEMM: Scheme $\langle 3,3,3 : 23 \rangle$ with 𝔽₂-search

Let's reproduce Laderman results for 3×3 matrices and rank 23.

In [5]:
# 0. generate corresponding tensor
%run scripts/generator.py gg 3 3 3

Saved: data\tensors\gg-333.npy and data\tensors\gg-333.meta.json  shape=(27, 4)  dtype=int8


In [6]:
# 1. run flip graph search (𝔽₂)
!{search2} gg-333 --plus --save

=== Pool-based Flip Graph Search ===
Tensor name: gg-333
Operation: gg
Dimensions: 3x3x3
Output ID: (none)
Tensor: data/tensors\gg-333.npy
Field: mod2
Dims: U=9, V=9, W=9  (nnz=27)
Path limit: 1000000, Pool size: 200, Target rank: 0, Stop attempts: 20000, Threads: 4, Plus transitions: enabled (limit: 50000)

Starting from rank 27

=== Searching for rank 26 ===
Completed 208 attempts in 2.6s
Found 208 candidate schemes of rank 26
Verified: 208/208

=== Searching for rank 25 ===
Completed 205 attempts in 2.3s
Found 205 candidate schemes of rank 25
Verified: 205/205

=== Searching for rank 24 ===
Completed 210 attempts in 2.4s
Found 210 candidate schemes of rank 24
Verified: 210/210

=== Searching for rank 23 ===
Completed 209 attempts in 3.0s
Found 207 candidate schemes of rank 23
Verified: 207/207

=== Searching for rank 22 ===
Completed 1001 attempts in 45.7s
Found 0 candidate schemes of rank 22
Failed to find schemes of rank 22 after 1001 attempts

=== Final Results ===
Best rank achi

In [7]:
# 2. run hensel lifting for 10 steps, to lift schemes from mod 2 to mod 2¹¹ algebra
#    run rational reconstruction to map mod 2¹¹ coefficients to ℤ or ℚ
!{lift2} gg-333 23

=== Hensel Lifting + Rational Reconstruction (mod 2) ===
Tensor name: gg-333
Operation:   gg
Shape:       (3, 3, 3)
Tensor:      data/tensors/gg-333.npy
Output ID:   (none)
Params:      nU=9, nV=9, nW=9
Rank:        23
Modulus:     2048 = 2^11
Threads:     8
Bound:       32
Verify:      no

Loaded 207 schemes (r=23)

Phase 1: Hensel lifting...
Progress: 50/207 (49 successful)
Progress: 100/207 (95 successful)
Progress: 150/207 (145 successful)
Progress: 200/207 (192 successful)
Progress: 207/207 (199 successful)


Lifting complete:
  Time:       0.05874 s
  Successful: 199/207

Saved lifted schemes to: data/schemes_lifted/gg-333/rank23-2pow11.npy
Format: (199, 621) uint64

Phase 2: Rational reconstruction...
Progress: 50/199 (50 successful)
Progress: 100/199 (100 successful)
Progress: 150/199 (150 successful)
Progress: 199/199 (199 successful)


Reconstruction complete:
  Time:       0.007151 s
  Successful: 199/199
  Integer:    192/199
  NNZ (min/avg/max): 162 / 247 / 354

Saved rati

In [8]:
# 3. select integer scheme with smallest number of additions
!{select2} gg-333 23

=== Recursion Analysis ===
Tensor name: gg-333
Operation: gg (symmetric)
Shape:     (3, 3, 3)
Rank:      23
Input dir: data/schemes_rational\gg-333
Files:     1 matching file(s)
Params:    nU=9, nV=9, nW=9

Loaded 199 rational schemes from 1 file(s)

Integer schemes:  192
Rational schemes: 7

  Unique triples (integer):  1
  Unique triples (rational): 1

  Pareto front (integer):  1 non-dominated triple(s)
  Pareto front (rational): 1 non-dominated triple(s)

Merging fronts...
  No rational schemes added (all dominated by or equal to integer front)

=== Final Pareto Front: 1 scheme ===
Triple           | Count   | Field   | NNZ    | MaxDenom  
------------------------------------------------------------
( 0, 0, 0)       | 199     | Z       | 162    | 1         

Saving selected schemes...
Saved 1 selected scheme (.npy + .txt) to data/schemes_selected/


In [9]:
# And here rank-23 scheme for 3x3 general matrix multiplication:
!cat data/schemes_selected/gg-333-rank23-rec-0-0-0-z.txt

m1  = (a1 - a5)(b2 - b4 + b6 + b8)
m2  = (a1 - a3)(b8)
m3  = (a5 - a8)(b2 + b5 + b8)
m4  = (a1 - a4 + a6)(b3 + b6 - b7 + b9)
m5  = (a8 - a9)(b6)
m6  = (a1 - a4)(b1 + b2 - b3 + b7 + b8 - b9)
m7  = (a1)(b1 + b4)
m8  = (a1 - a2 - a7 + a8)(b4 + b7)
m9  = (a4 - a5 - a7 + a8)(b2)
m10 = (a1 - a2 + a3 + a8 - a9)(b7)
m11 = (a6)(b3 + b6 + b9)
m12 = (a7)(b1 + b4 + b7)
m13 = (a1 - a3 - a4 + a6)(b7 + b8 - b9)
m14 = (a5 - a6 - a8 + a9)(b8)
m15 = (a1 - a2 + a8 - a9)(b6 + b7)
m16 = (a5)(b2 + b5 + b8)
m17 = (a4 - a6)(b3 + b6 + b8)
m18 = (a2 - a5)(b4 + b5 - b6)
m19 = (a4 - a5)(b2 + b6 + b8)
m20 = (a1 - a2)(b4 - b6)
m21 = (a6 - a9)(b6 + b9)
m22 = (a6 - a7)(b3)
m23 = (a4 - a5 + a8 - a9)(b6 + b8)

c1 = m5 + m7 + m10 - m15 - m20
c2 = m1 - m2 + m16 + m18 + m20
c3 = -m2 + m4 + m5 + m10 + m13 - m15 + m17
c4 = m1 - m4 - m6 + m7 + m11 - m19
c5 = m5 - m14 + m16 + m19 - m23
c6 = m5 + m11 - m14 + m17 - m23
c7 = m5 + m8 + m12 - m15 - m20
c8 = -m3 + m5 - m9 + m16 + m19 - m23
c9 = m5 + m11 - m21 - m22


In [10]:
# We can check output of the scheme, substituting m and expanding them
%run scripts/print_scheme_output.py data/schemes_selected/gg-333-rank23-rec-0-0-0-z.txt

c1 = a1*b1 + a2*b4 + a3*b7
c2 = a1*b2 + a2*b5 + a3*b8
c3 = a1*b3 + a2*b6 + a3*b9
c4 = a4*b1 + a5*b4 + a6*b7
c5 = a4*b2 + a5*b5 + a6*b8
c6 = a4*b3 + a5*b6 + a6*b9
c7 = a7*b1 + a8*b4 + a9*b7
c8 = a7*b2 + a8*b5 + a9*b8
c9 = a7*b3 + a8*b6 + a9*b9


So we have correct calculations:
$$
\begin{pmatrix}
c_1 & c_2 & c_3 \\
c_4 & c_5 & c_6 \\
c_7 & c_8 & c_9
\end{pmatrix}
=
\begin{pmatrix}
a_1 & a_2 & a_3 \\
a_4 & a_5 & a_6 \\
a_7 & a_8 & a_9
\end{pmatrix}
\begin{pmatrix}
b_1 & b_2 & b_3 \\
b_4 & b_5 & b_6 \\
b_7 & b_8 & b_9
\end{pmatrix}
=
\begin{pmatrix}
a_1 b_1 + a_2 b_4 + a_3 b_7 & a_1 b_2 + a_2 b_5 + a_3 b_8 & a_1 b_3 + a_2 b_6 + a_3 b_9 \\
a_4 b_1 + a_5 b_4 + a_6 b_7 & a_4 b_2 + a_5 b_5 + a_6 b_8 & a_4 b_3 + a_5 b_6 + a_6 b_9 \\
a_7 b_1 + a_8 b_4 + a_9 b_7 & a_7 b_2 + a_8 b_5 + a_9 b_8 & a_7 b_3 + a_8 b_6 + a_9 b_9
\end{pmatrix}.
$$

### Complex Scheme $\langle 2,2,2 : 20 \rangle$ with 𝔽₂-search (𝔽₂-scheme)

How many real multiplications we need to multiply two complex 2x2 matrices? Using gauss trick
$$
(a_1 + i a_2)(b_1 + i b_2)
= 
(a_1 b_1 - a_2 b_2)
+ \bigl((a_1 + a_2)(b_1 + b_2) - a_1 b_1 - a_2 b_2\bigr),
$$
and Strassen scheme we can do it in 21 multiplications. But we can also do it in 20 in 𝔽₂.

In [11]:
%%capture

# 0. generate corresponding tensor
%run scripts/cgenerator.py ggc 2 2 2

# 1. run flip graph search (𝔽₃)
!{search2} ggc-222 --plus --save

# 2. select modular scheme
!{select_modular2} ggc-222 20

In [12]:
!cat data/schemes_selected/ggc-222-rank20-f2.txt

m1  = (a3 + a8)(b3 + b4 + b8)
m2  = (a1 + a2 + a5)(b1 + b3 + b5 + b7)
m3  = (a2 + a3 + a4 + a6)(b3)
m4  = (a2 + a3 + a5 + a6 + a7)(b2 + b6)
m5  = (a2 + a6 + a7)(b2 + b4 + b6 + b8)
m6  = (a1 + a2 + a3 + a4 + a5 + a6 + a7 + a8)(b1 + b2 + b4 + b5 + b8)
m7  = (a1 + a2 + a3 + a5 + a7)(b1 + b3 + b4 + b5 + b7 + b8)
m8  = (a1 + a2 + a4 + a5)(b1 + b4 + b8)
m9  = (a2 + a4 + a6 + a8)(b1 + b2 + b5 + b7 + b8)
m10 = (a1 + a7)(b2 + b3 + b5 + b6 + b7)
m11 = (a2 + a6)(b2 + b4 + b6)
m12 = (a1 + a2 + a4 + a5 + a8)(b4 + b8)
m13 = (a1 + a2 + a3 + a4)(b1 + b3 + b7)
m14 = (a3 + a4 + a7 + a8)(b3 + b4 + b7 + b8)
m15 = (a3 + a4)(b1 + b7)
m16 = (a1 + a5)(b4 + b6 + b8)
m17 = (a1 + a2 + a5 + a7)(b3 + b4 + b5 + b7 + b8)
m18 = (a2 + a5)(b3 + b5 + b7)
m19 = (a3 + a7)(b1 + b2 + b3 + b4 + b5 + b6 + b7 + b8)
m20 = (a2 + a3 + a4 + a6 + a7 + a8)(b1 + b2 + b3 + b5 + b7)

c1 = m3 + m4 + m5 + m6 + m7 + m10 + m13 + m14 + m15 + m16 + m17 + m19 + m20
c2 = m5 + m10 + m11 + m16 + m17 + m18
c3 = m1 + m2 + m8 + m14 + m15 + m17
c4 =

In [13]:
%run scripts/print_scheme_output.py --mod 2 data/schemes_selected/ggc-222-rank20-f2.txt

c1 = a1*b1 + a2*b3 + a5*b5 + a6*b7
c2 = a1*b2 + a2*b4 + a5*b6 + a6*b8
c3 = a3*b1 + a4*b3 + a7*b5 + a8*b7
c4 = a3*b2 + a4*b4 + a7*b6 + a8*b8
c5 = a1*b5 + a2*b7 + a5*b1 + a6*b3
c6 = a1*b6 + a2*b8 + a5*b2 + a6*b4
c7 = a3*b5 + a4*b7 + a7*b1 + a8*b3
c8 = a3*b6 + a4*b8 + a7*b2 + a8*b4


Corresponding to
$$
\begin{pmatrix}
c_1 + i c_5 & c_2 + i c_6 \\
c_3 + i c_7 & c_4 + i c_8
\end{pmatrix}
=
\begin{pmatrix}
a_1 + i a_5 & a_2 + i a_6 \\
a_3 + i a_7 & a_4 + i a_8
\end{pmatrix}
\begin{pmatrix}
b_1 + i b_5 & b_2 + i b_6 \\
b_3 + i b_7 & b_4 + i b_8
\end{pmatrix}
=
\begin{pmatrix}
a_1 b_1 + a_2 b_3 - a_5 b_5 - a_6 b_7
+ i\left(a_1 b_5 + a_2 b_7 + a_5 b_1 + a_6 b_3\right)
&
a_1 b_2 + a_2 b_4 - a_5 b_6 - a_6 b_8
+ i\left(a_1 b_6 + a_2 b_8 + a_5 b_2 + a_6 b_4\right)
\\[4pt]
a_3 b_1 + a_4 b_3 - a_7 b_5 - a_8 b_7
+ i\left(a_3 b_5 + a_4 b_7 + a_7 b_1 + a_8 b_3\right)
&
a_3 b_2 + a_4 b_4 - a_7 b_6 - a_8 b_8
+ i\left(a_3 b_6 + a_4 b_8 + a_7 b_2 + a_8 b_4\right)
\end{pmatrix},
$$
calculated in 𝔽₂ with 20 real multiplications.

## Structured matrix multiplication

### SYRK: Scheme $\langle 3,3,3 : 17 \rangle_{\texttt{gt}}^{(9,0,0)}$ with 𝔽₂-search

In progress.

### Scheme $\langle 2,2,2 : 5 \rangle_{\texttt{ss}}^{(0,2,0)}$ with 𝔽₃-search

In [14]:
%%capture

# 0. generate corresponding tensor
%run scripts/generator.py ss 2 2 2

# 1. run flip graph search (𝔽₃)
!{search3} ss-222 --plus --save

# 2. run hensel lifting and rational reconstruction
!{lift3} ss-222 5

# 3. select rational scheme
!{select2} gg-333 23

In [15]:
!cat data/schemes_selected/ss-222-rank5-rec-0-2-0-q.txt

m1 = (1/2a1 + 1/2a2)(b1 + b2)
m2 = (a1 - a3)(b2)
m3 = (a2)(b1 - b3)
m4 = (1/2a1 - 1/2a2)(b1 - b2)
m5 = (a2 - a3)(b2 - b3)

c1 = m1 + m4
c2 = m1 - m3 - m4
c3 = m1 - m2 - m4
c4 = m1 - m2 - m3 - m4 + m5


In [16]:
%run scripts/print_scheme_output.py data/schemes_selected/ss-222-rank5-rec-0-2-0-q.txt

c1 = a1*b1 + a2*b2
c2 = a1*b2 + a2*b3
c3 = a2*b1 + a3*b2
c4 = a2*b2 + a3*b3


So these calculations
$$
\begin{pmatrix}
c_1 & c_2 \\
c_3 & c_4
\end{pmatrix}
=
\begin{pmatrix}
a_1 & a_2 \\
a_2 & a_3
\end{pmatrix}
\begin{pmatrix}
b_1 & b_2 \\
b_2 & b_3
\end{pmatrix}
=
\begin{pmatrix}
a_1 b_1 + a_2 b_2 & a_1 b_2 + a_2 b_3 \\
a_2 b_1 + a_3 b_2 & a_2 b_2 + a_3 b_3
\end{pmatrix}
$$
can be done in 5 multiplications, instead naive 8.

In [25]:
8 * 3 * 20

480

In [29]:
9*3*23 

621

In [26]:
np.load("data/schemes_selected/gg-333-rank23-rec-0-0-0-z.npy").shape

(1, 1242)

In [35]:
np.load("data/schemes_selected/ss-222-rank5-rec-0-2-0-q.npy")[0][1::2]

array([2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1], dtype=int64)